### Import Libraries

In [1]:
import yaml
import re
from api_search import serp_google_search, google_cse_search
import json
import pandas as pd
from datetime import datetime
import requests
import math
from serpapi import GoogleSearch
import json

with open("config.yml", "r") as file:
    api_keys = yaml.safe_load(file)


### Google Customizable Search

In [2]:
query = "Azuria Water Solutions acquisition 2022 OR 2023 OR 2024 OR 2025"

url = "https://www.googleapis.com/customsearch/v1"

results_google = google_cse_search(query, api_keys['api_key']
                            , cse_id= api_keys['search_engine_id'], total_results=20)


In [3]:
params = {
        'key': api_keys['api_key'],
        'cx': api_keys['search_engine_id'],
        'q': query,
        'num': 10
    }

response = requests.get(url, params=params)
res_cse = response.json()
# results = []

# for item in data.get('items', []):
#     results.append({
#         'title': item.get('title'),
#         'link': item.get('link'),
#         'snippet': item.get('snippet')
#     })

# return results

In [4]:
res_cse['items']

[{'kind': 'customsearch#result',
  'title': 'Azuria Water Solutions Announces the Acquisition of TSW Utility ...',
  'htmlTitle': '<b>Azuria Water Solutions</b> Announces the <b>Acquisition</b> of TSW Utility ...',
  'link': 'https://www.azuria.com/news/2025/april/azuria-acquires-tsw-utility-solutions-am-liner-east-and-ck-masonry',
  'displayLink': 'www.azuria.com',
  'snippet': 'LOUIS (April 21, 2025) – Azuria Water Solutions™ the leading provider of infrastructure maintenance, rehabilitation and technology-enabled water solutions,\xa0...',
  'htmlSnippet': 'LOUIS (April 21, <b>2025</b>) – <b>Azuria Water Solutions</b>™ the leading provider of infrastructure maintenance, rehabilitation and technology-enabled <b>water solutions</b>,&nbsp;...',
  'formattedUrl': 'https://www.azuria.com/.../2025/.../azuria-acquires-tsw-utility-solutions-am-...',
  'htmlFormattedUrl': 'https://www.<b>azuria</b>.com/.../<b>2025</b>/.../<b>azuria</b>-acquires-tsw-utility-<b>solutions</b>-am-...',
  'pagemap

### Serp API

In [5]:
params = {
  "api_key": api_keys["serp_api_key"],  "engine": "google",
  "q": query,  "location": "Austin, Texas, United States",
  "google_domain": "google.com",  "gl": "us",
  "hl": "en",  "num":25
}
nws=False
if nws:
    params['tbm'] = "nws"
# tbm:"nws"

search_serp = GoogleSearch(params)
results_serp = search_serp.get_dict()


In [6]:
google_results_serp1 = [{x:y for x,y in z.items() if x in ['title', 'link', 'snippet', 'formattedUrl']} for z in results_serp['organic_results']]

In [7]:
results_google

[{'title': 'Azuria Water Solutions Announces the Acquisition of TSW Utility ...',
  'link': 'https://www.azuria.com/news/2025/april/azuria-acquires-tsw-utility-solutions-am-liner-east-and-ck-masonry',
  'snippet': 'LOUIS (April 21, 2025) – Azuria Water Solutions™ the leading provider of infrastructure maintenance, rehabilitation and technology-enabled water solutions,\xa0...'},
 {'title': 'Azuria Water Solutions Announces the Acquisition of TSW',
  'link': 'https://www.globenewswire.com/news-release/2025/04/21/3064780/0/en/Azuria-Water-Solutions-Announces-the-Acquisition-of-TSW-Utility-Solutions-Inc-AM-Liner-East-Inc-and-C-K-Masonry-Company-Inc.html',
  'snippet': 'Apr 21, 2025 ... Masonry Company, Inc. A Leader of Technology-Enabled Water Solutions Has Welcomed Three Companies This Year. April 21, 2025 11:17 ET\xa0...'},
 {'title': 'Azuria Water Solutions: Over 50 Years of Innovation',
  'link': 'https://www.azuria.com/',
  'snippet': 'Azuria has been making infrastructure safer and s

In [8]:
pd.set_option('display.max_colwidth', None)  # or a large number like 200
serp_df = pd.DataFrame(google_results_serp1)
cse_df = pd.DataFrame(results_google)

serp_df['rank_serp'] = [x+1 for x in range(serp_df.shape[0])]
cse_df['rank_cse'] = [x+1 for x in range(cse_df.shape[0])]

serp_df.to_csv("serp_output.csv", index=False)
cse_df.to_csv("cse_output.csv", index=False)

In [9]:
for x,y in zip(results_google[0:20], google_results_serp1[0:20]):
    print("------------------------------------------------------------")
    print(f"CSE Snippet : {y['snippet']} \n SERP Snippet: {x['snippet']}")

------------------------------------------------------------
CSE Snippet : Azuria acquires TSW Utility Solutions, AM-Liner East., and C.K. Masonry Company. 
 SERP Snippet: LOUIS (April 21, 2025) – Azuria Water Solutions™ the leading provider of infrastructure maintenance, rehabilitation and technology-enabled water solutions, ...
------------------------------------------------------------
CSE Snippet : Azuria Water Solutions has acquired BLD Services, LLC, of Kenner, Louisiana.... 
 SERP Snippet: Apr 21, 2025 ... Masonry Company, Inc. A Leader of Technology-Enabled Water Solutions Has Welcomed Three Companies This Year. April 21, 2025 11:17 ET ...
------------------------------------------------------------
CSE Snippet : Azuria Announces Acquisition of Infrastructure Services Group LLC and Subsidiaries · Azuria Announces the Acquisition of Oxbow Construction · Aegion Rebrands as ... 
 SERP Snippet: Azuria has been making infrastructure safer and stronger by rehabilitating and maintain

| Feature                        | **SerpAPI**                               | **Google CSE JSON API**                            |
| ------------------------------ | ----------------------------------------- | -------------------------------------------------- |
| **Result Accuracy**            | ✅ Matches real-time Google SERP           | ⚠️ Not identical, uses Programmable Search backend |
| **Real-Time Freshness**        | ✅ Yes, real-time scrape                   | ❌ May lag behind actual SERP                       |
| **Supports Ads, Images, News** | ✅ Yes (Top stories, ads, maps, etc.)      | ❌ No (text results only)                           |
| **Filtering (e.g. by date)**   | ✅ Advanced filters available              | ⚠️ Limited filters                                 |
| **Custom Site Bias**           | ❌ Neutral (true SERP)                     | ✅ Can emphasize domains or restrict to them        |
| **Configuration Needed**       | ✅ Simple (just query)                     | ⚠️ Needs setup (CSE ID, config)                    |
| **Quota**                      | ⚠️ Paid (starts \$50/mo, generous limits) | ✅ Free up to 100 queries/day, then \$5 per 1000    |
| **Legal/Compliance**           | ⚠️ Scrapes Google – may violate ToS       | ✅ Official Google-supported API                    |
| **Pagination**                 | ✅ Supports multiple pages                 | ✅ Supports via `start` param                       |
| **Ease of Integration**        | ✅ Very easy (just URL and params)         | ✅ Easy, but config-dependent                       |
| **Rate Limit Handling**        | ✅ Built-in rotating proxies               | ❌ Strict limits                                    |
| **Use Case Fit**               | Real-time analysis, scraping, SEO tools   | Educational, lightweight apps, low-volume search   |
